# How to calculate the disconnected (= Gaussian) power spectrum covariance for cutsky geometry

Reference: [Yin Li+18](https://arxiv.org/abs/1811.05714).

## Given an input theory $P(k)$

In [ ]:
from matplotlib import pyplot as plt

from jax import numpy as jnp
from jaxpower import MeshAttrs, Mesh2SpectrumPole, Mesh2SpectrumPoles, BinMesh2SpectrumPoles, compute_spectrum2_covariance

def get_theory(kmax=0.3, dk=0.005):
    # Return theory power spectrum
    from cosmoprimo.fiducial import DESI
    cosmo = DESI(engine='eisenstein_hu')
    z = 1.
    pk1d = cosmo.get_fourier().pk_interpolator().to_1d(z=z)
    ellsin = (0, 2, 4)
    # Theory k-bins
    edgesin = jnp.arange(0., kmax, dk)
    edgesin = jnp.column_stack([edgesin[:-1], edgesin[1:]])
    kin = jnp.mean(edgesin, axis=-1)
    f, b = cosmo.growth_rate(z), 1.5
    beta = f / b
    shotnoise = (1e-3)**(-1)
    pk = pk1d(kin)

    poles = jnp.array([(1. + 2. / 3. * beta + 1. / 5. * beta ** 2) * pk
                        (4. / 3. * beta + 4. / 7. * beta ** 2) * pk,
                        8. / 35 * beta ** 2 * pk])
    theory = []
    for ell, value in zip(ellsin, poles):
        theory.append(ObservableLeaf(k=kin, k_edges=edgesin, value=value, meta=dict(ell=ell)))
    return ObservableTree(theory, ells=ellsin)

In [ ]:
mattrs = MeshAttrs(boxsize=2000., boxcenter=[0., 0., 2200], meshsize=128)

def generate_survey_sample(size, seed, paint=False):
    # Generate a collection of particles with the survey geometry
    from jaxpower import ParticleField
    # Generate Gaussian-distributed positions
    positions = random.normal(seed, shape=(size, 3))
    mask = jnp.all((positions > -1.) & (positions < 1.), axis=-1)
    positions = positions * 0.25 * mattrs.boxsize + mattrs.boxcenter
    toret = ParticleField(positions, weights=1. * mask, attrs=mattrs)
    if paint: toret = toret.paint(resampler='cic', interlacing=0, compensate=False)
    return toret


In [ ]:
# To compute the window functions for the covariance matrix,
# better use randoms that represent the noise in the data (similar scatter of weights, if any)
# Of course, you can use more randoms, and provide alpha = (number of data) / (number of randoms)
size = int(1e6)  # number of data points
alpha = 0.1  # (number of data) / (number of randoms)
randoms = generate_survey_sample(int(size / alpha), random.key(42), paint=False)

windows = compute_fkp2_covariance_window(randoms, alpha=alpha, edges={'step': mattrs.cellsize.min()},
                                         interlacing=2, resampler='tsc', los='local')
# Then compute the covariance matrix
# delta is the maximum abs(k1 - k2) where the covariance will be computed (to speed up calculation)
covs_analytical = compute_spectrum2_covariance(windows, theory, delta=0.2)

# Sum all contributions (WW, WS, SS), with W = standard window (multiplying delta), S = shotnoise
cov_analytical = covs_analytical[0].clone(value=sum(cov.value() for cov in covs_analytical))
cov_analytical.plot(show=True);

In [ ]:
from jaxpower import FKPField, compute_fkp2_normalization, compute_fkp2_shotnoise

theory = get_theory(kmax=np.sqrt(3) * mattrs.knyq.max())
bin = BinMesh2SpectrumPoles(mattrs, edges={'step': 0.01}, ells=(0, 2, 4))
los = 'local'

@jit
def generate_mock(seed, theory=theory):
    seeds = random.split(seed)
    mesh = generate_anisotropic_gaussian_mesh(mattrs, poles=theory, los=los, seed=seed)
    data = generate_survey_sample(size, seeds[0], paint=False)
    data = data.clone(weights=1. + mesh.read(data.positions, resampler='cic', compensate=True))
    randoms = generate_survey_sample(5 * size, seeds[1], paint=False)
    fkp = FKPField(data, randoms)
    mesh = fkp.paint(resampler='tsc', interlacing=3, compensate=True, out='complex')
    norm = compute_fkp2_normalization(fkp, bin=bin, cellsize=None)
    num_shotnoise = compute_fkp2_shotnoise(fkp, bin=bin)
    return compute_mesh2_spectrum(mesh, bin=bin, los=los).clone(norm=norm, num_shotnoise=num_shotnoise)

from tqdm import tqdm
mocks = []
for imock in tqdm(range(100)):
    mocks.append(generate_mock(random.key(imock)))

cov_mocks = types.cov(mocks)
cov_mocks.plot(show=True);

In [ ]:
# data shot noise of these Gaussian mocks is non trivial because of the 1 + delta weights
# So let's just rescale the shot noise part of the covariance matrix
with_delta = generate_mock(random.key(0)).get(ells=0).values('shotnoise')[0]
without_delta = generate_mock(random.key(0), theory=theory.clone(value=0. * theory.value())).get(ells=0).values('shotnoise')[0]
ratio_shotnoise = cov_mocks.observable.get(ells=0).values('shotnoise')[0] / cov_mocks_shotnoise.observable.get(ells=0).values('shotnoise')[0]

ytransform = lambda x, y: x**2 * y
kw = dict(ytransform=ytransform)
cov_ws = covs_analytical[1].clone(value=ratio_shotnoise * covs_analytical[1].value())
cov_ss = covs_analytical[2].clone(value=ratio_shotnoise**2 * covs_analytical[2].value())
fig = covs[0].clone(value=covs[0].value() + cov_ws.value() + cov_ss.value()).plot_diag(**kw, color='C0')
cov_mocks.plot_diag(**kw, color='C1', fig=fig, show=True)